In [19]:
# Import libraries
import pandas as pd
import numpy as np
import pyodbc

### CONNECTION STRING ###
# remove details prior to upload. Add details before use
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                          'Server=<ServerName>;Database=<DatabaseName>;'
                          'Trusted_Connection=<ConnectionType>;')
###

# import seaborn as sns
# from matplotlib import pyplot as plt

# Instantiate a Clinical Trial Object

In [20]:
class clinical_trial:
    def __init__(self,studynum,visits,cohort,specimen_types,exceptions):
        self.studynum = studynum
        self.visits = visits
        self.cohort = cohort
        self.specimen_types = specimen_types
        self.exceptions = exceptions


    
reboot = clinical_trial("ITN080AI", #studynum
                    {'A':['0A', '1A', '2A', '3A', '4A', '7A', '10A', '13A', '17A', '18A', '19A', '20A', '21A', '22A', '23A', '24A', '25A','DVA'],
                    "B":['0B', '1B', '3B', '5B', '7B', '9B', '10B', '11B', '12B', '13B', '14B', '15B', '16B', '17B','DVB'],
                    }, # Visit dictionary
                    ["A","B"], # cohort
                    {"Serum Clot":["H"+"%02d" % i for i in list(range(1,24))],
                  "PBMC":['10A','10B','10C','10D','10E','10F','10G','10H','10I'],
                  "Whole Blood\nTranscriptomics":['9A','9B'],
                  "Whole Blood\nEpigenetic":['51','52','53','54'],
                "Urine Supernatant":["D"+"%02d" % i for i in list(range(1,11))],
                "Urine Pellet":['R01','R02']
                  }, # spectype dict
                    {"A":[('PBMC','1A'),('PBMC','2A'),('PBMC','3A'),
                          ('Urine Pellet','1A'),('Urine Pellet','2A'),('Urine Pellet','3A'),
                          ('Urine Supernatant','1A'),('Urine Supernatant','2A'),('Urine Supernatant','3A'),
                          ('Whole Blood\nEpigenetic','1A'),('Whole Blood\nEpigenetic','2A'),('Whole Blood\nEpigenetic','3A'),
                          ('Whole Blood\nTranscriptomics','1A'),('Whole Blood\nTranscriptomics','2A'),('Whole Blood\nTranscriptomics','3A')]},# exceptions

                    ) 

reveal = clinical_trial("ITN086AI", # Studynum
                    ['0','3','6','8','9','10','WD'], # visits
                    None, # cohort?
                    {"Skin Biopsy":["M01","M02"],
                    "Whole Blood RNA":['9A','9B','9C'],
                    "Whole Blood DNA":['51','52','53','54','55','56'],
                    "PBMC":["10A","10B","10C","10D","10E","10F"],
                    "Serum":['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10']}, # spec type dict
                     [('Skin Biopsy','3'),('Skin Biopsy','9')] # exceptions
                    )

graduate = clinical_trial("ITN084AD", # Studynum
                    ['-2','-1','0',"S1","S3","S4",'S5', 'S6','S8', 'S9','S10', 'S11',"S13",'S14','S15'], # visits
                    None, # cohorts?
                    {'Nasal Brushing':0,'Nasal Fluid':0,'PBMC-Li Hep':0, 'Plasma-Li Hep':0, 'Serum-Clot':0,  'Whole Blood':0}, # GRADUATE specimen types granular enough. values not needed

                    [("PBMC-Li Hep","-1"),("Plasma-Li Hep","-1"),
                            ("Nasal Brushing","S1"),("PBMC-Li Hep","S1"),("Plasma-Li Hep","S1"),("Whole Blood","S1"),
                            ("PBMC-Li Hep","S4"),("Plasma-Li Hep","S4"),
                            ("Nasal Brushing","S6"),("PBMC-Li Hep","S6"),("Plasma-Li Hep","S6"),("Whole Blood","S6"),
                            ("PBMC-Li Hep","S9"),("Plasma-Li Hep","S9"),
                            ("Nasal Brushing","S11"),("PBMC-Li Hep","S11"),("Plasma-Li Hep","S11"),("Whole Blood","S11"),
                            ("PBMC-Li Hep","S14"),("Plasma-Li Hep","S14"),
                            ] # exceptions
                    )

clinical_trials = [reboot,reveal,graduate]

In [21]:
# Get data from LabVantage. Connect to the SQL Server.
# If there are cohorts get data for the specified cohort
# Then, keep relevant columns and drop samples that are lost/damaged/missing.
# Run data cleaning functions specific to the study.
# 
def get_lv_data(ct,studynum, cohort):
    
    lv_query = None
    if cohort == None:
        lv_query = '''SELECT * FROM rpt.LabVantageVisits WHERE studynum = '{}' '''.format(studynum)
    else:
        lv_query = '''SELECT * FROM rpt.LabVantageVisits WHERE studynum = '{}' AND Cohort = '{}' '''.format(studynum,cohort)
    output_df = pd.read_sql(lv_query,cnxn)

    output_df = output_df[['studynum',  'Participant', 'KitBarcode',
                            'CollectionDate',  'specimentype',
                            'visitnum', 'barcode', 'Sample Comment', 'Cohort', 'storagestatus',
                            'storagedisposalstatus', 'Shipping Status']]
    output_df = output_df[~output_df["storagedisposalstatus"].isin(['lostOrDamaged', 'Missing'])]
    if ct.studynum == "ITN080AI":
        output_df.drop(output_df[(output_df["barcode"] == "340236")].index, inplace = True)
    
    # Rename and specify sample types only for non-graduate
    if ct.studynum != "ITN084AD":
        def specify_sample_type(bcde):
            # Turn the barcode into a list where the first entry is the kit and the second entry is the suffix
            bcde_components = bcde.split("-")
            # Get the suffix
            suffix = bcde_components[1]
            
            specimen_type_keys = list(ct.specimen_types.keys())
            #print(suffix)
            for s_t in specimen_type_keys:
                if suffix in ct.specimen_types[s_t]:
                    return s_t
            return None

        output_df["specimentype"] = output_df.apply(lambda x:specify_sample_type(x["barcode"]),axis = 1)

    # custom data cleaning for graduate:
    if ct.studynum == "ITN084AD":
        def assign_visit(kit,vis):
            if vis == None:
                if kit == "447410":
                    return "S3"
                elif kit == "715208":
                    return "0"
                elif kit == "822140":
                    return "-2"
                else:
                    return vis
            else:
                return vis
            
        def assign_pid(kit,pid):
            if pid == None:
                if kit == "447410":
                    return "11651"
                elif kit == "715208":
                    return "11539"
                elif kit == "822140":
                    return "11302"
                else:
                    return pid
            else:
                return pid
            
        output_df["visitnum"] = output_df.apply(lambda x: assign_visit(x["KitBarcode"],x["visitnum"]),axis = 1)
        output_df["Participant"] = output_df.apply(lambda x: assign_pid(x["KitBarcode"],x["Participant"]),axis = 1)        

    return output_df

# Get Rho data
def get_rho_data(ct,studynum,cohort):


    # If there is/n't multiple cohorts for the study
    if ct.cohort != None:
        rho_query = '''SELECT DISTINCT a.[ADINFC STUDYID],a.[RHO Screening Identifier],a.[Cohort],a.[Participant ID],c.VisitKey, c.[Visit Number],c.[Visit Ordinal],c.[DaysPostScreening]
                    FROM   [rpt].[Participant] a
                    JOIN   [rpt].[ParticipantActivity] b
                        ON     a.[ParticipantKey] = b.[ParticipantKey]
                        AND    b.[Activity] IN ('Visit','UnscheduledVisit')
                    JOIN   [rpt].[Visit] c
                        ON     b.[VisitKey] = c.[VisitKey]
                    JOIN   [rpt].[Site] d
                        ON     a.[SiteKey] = d.[SiteKey]
                    WHERE  a.[ADINFC STUDYID] = '{}' AND Cohort = 'Part {}' '''.format(studynum,cohort)
    else:
        rho_query = '''SELECT DISTINCT a.[ADINFC STUDYID],a.[RHO Screening Identifier],a.[Cohort],a.[Participant ID], c.VisitKey, c.[Visit Number],c.[Visit Ordinal],c.[DaysPostScreening]
                    FROM   [rpt].[Participant] a
                    JOIN   [rpt].[ParticipantActivity] b
                        ON     a.[ParticipantKey] = b.[ParticipantKey]
                        AND    b.[Activity] IN ('Visit','UnscheduledVisit')
                    JOIN   [rpt].[Visit] c
                        ON     b.[VisitKey] = c.[VisitKey]
                    JOIN   [rpt].[Site] d
                        ON     a.[SiteKey] = d.[SiteKey]
                    WHERE  a.[ADINFC STUDYID] = '{}'  '''.format(studynum)
    # Turn query results into a dataframe
    output_df = pd.read_sql(rho_query,cnxn)
    


    # Fix the lack of pids
    def fix_no_pid_rho(rho_si,pid):
        if pid == None:
            rho_si_components = rho_si.split("-")
            return rho_si_components[2]
        else:
            return pid
    output_df["Participant ID"] = output_df.apply(lambda x: fix_no_pid_rho(x["RHO Screening Identifier"],x["Participant ID"]),axis = 1)
    return output_df


def get_visit_info(ct):

    visit_query = '''SELECT a.[Study Number],b.[Visit Number],b.[Visit Ordinal],b.[DaysPostScreening] FROM rpt.Study a
                    JOIN rpt.Visit b
                        ON a.[StudyKey] = b.[Studykey]
                        WHERE a.[Study Number] = '{}'
                        ORDER BY [Visit Ordinal]
                    '''.format(ct.studynum)
    output_df = pd.read_sql(visit_query,cnxn)

    return output_df

# Create a result dataframe
def create_result_df(ct,choose_cohort,rho_records,lv_data,visit_info):
    # Create the resulting dataframe
    result = pd.DataFrame(columns = ["Study","Cohort","Visit Number","Visit Ordinal","DaysPostScreening","Sample Type","Number at least 1 collected","Number of recorded visits","Percent"])
    
    visit_ord_dict = dict(zip(list(visit_info["Visit Number"]),list(visit_info["Visit Ordinal"])))
    days_post_dict = dict(zip(list(visit_info["Visit Number"]),list(visit_info["DaysPostScreening"])))
    
    if type(ct.cohort) == list:
        for visit in ct.visits[choose_cohort]:
            for sampletype in ct.specimen_types:
                result.loc[len(result)] = [ct.studynum,choose_cohort,visit,visit_ord_dict[visit],days_post_dict[visit],sampletype,None,None,None]
    else:
        for visit in ct.visits:
            for sampletype in ct.specimen_types:
                result.loc[len(result)] = [ct.studynum,None,visit,visit_ord_dict[visit],days_post_dict[visit],sampletype,None,None,None]
        

    number_of_recorded_visits = []
    def find_collection_stats(std,visnum,spectype):
        # Find the number of PIDs with a visit recorded
        rho_visits = rho_records[rho_records["Visit Number"] == visnum]
        pids_with_recorded_visit_rho = len(list(rho_visits["Participant ID"].unique()))
        pids_with_recorded_visit_lv = len(list(lv_data[lv_data["visitnum"] == visnum]["Participant"].unique()))
        pids_with_recorded_visit = max(pids_with_recorded_visit_rho,pids_with_recorded_visit_lv)
        #if pids_with_recorded_visit_rho != pids_with_recorded_visit_lv:
        #    print(ct.studynum + " " + visnum + ": RHO has {} visits recorded but LV has {} visits recorded".format(str(pids_with_recorded_visit_rho),str(pids_with_recorded_visit_lv)))
        
        # LV data with the specimen type and visit num
        lv_data_visit_spec = lv_data[(lv_data["specimentype"] == spectype) & (lv_data["visitnum"] == visnum)]

        # Number of unique pids that have the spec type for that visit
        pids_at_least_1_tube_for_this_spec_type_for_this_vis = len(list(lv_data_visit_spec["Participant"].unique()))

        percent = 0

        if pids_with_recorded_visit == 0:
            percent = 0
        else:
            percent = 100 * pids_at_least_1_tube_for_this_spec_type_for_this_vis / pids_with_recorded_visit
        
        number_of_recorded_visits.append(pids_with_recorded_visit)

        #print(spectype + " | " + visnum + ": " + str(pids_at_least_1_tube_for_this_spec_type_for_this_vis))

        return pids_at_least_1_tube_for_this_spec_type_for_this_vis, pids_with_recorded_visit,round(percent)

    result[["Number at least 1 collected","Number of recorded visits","Percent"]] = result.apply(lambda x: find_collection_stats(x["Study"],x["Visit Number"],x["Sample Type"]),axis = 1,result_type="expand")
    
    return result

# Create plot
def heatmap(ct,choose_cohort,table,table_num_rec_vis):
    # Create a new list of xlabels:
    num_visits_per_vis = table_num_rec_vis.iloc[0,:]
    visit_labels =[]

    if type(ct.cohort) == list:
        for ind in range(len(ct.visits[choose_cohort])):
            visit_labels.append("V"+ct.visits[choose_cohort][ind]+"\n"+str(num_visits_per_vis[ind]))
            if ind == len(ct.visits)-1 & (ct.studynum != "ITN084AD"):
                visit_labels[ind] = ct.visits[choose_cohort][ind]+"\n"+str(num_visits_per_vis[ind])
    else:
        for ind in range(len(ct.visits)):
            visit_labels.append("V"+ct.visits[ind]+"\n"+str(num_visits_per_vis[ind]))
            if ind == len(ct.visits)-1 & (ct.studynum != "ITN084AD"):
                visit_labels[ind] = ct.visits[ind]+"\n"+str(num_visits_per_vis[ind])
                
    title = ct.studynum + ": % of PIDs that have at least 1 Sample Collected for the Visit"
    plt.figure(figsize=(15,5))
    cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True)
    fig = sns.heatmap(table,annot= True,fmt='g',cmap=cmap,vmin = 0 ,vmax = 100)
    fig.set_xticklabels(visit_labels, rotation=0)
    fig.set_yticklabels(list(table.index), rotation=0)
    fig.xaxis.tick_top()
    fig.set(xlabel='Visit',
        ylabel='Specimen Type',title=title)
    
# Get collection statistics
def pivot_table(ct,choose_cohort,rho_records,res,lv_data):
    number_of_recorded_visits = []

    # make a pivot table to get the number of recorded visits
    table_num_rec_vis = pd.pivot_table(res,values="Number of recorded visits",index = ["Sample Type"],columns=["Visit Number"])

    # re-index the table based on the ordered list of visits
    if type(ct.cohort) == list:
        table_num_rec_vis = table_num_rec_vis.reindex(ct.visits[choose_cohort],axis = 1)
    else:
        table_num_rec_vis = table_num_rec_vis.reindex(ct.visits,axis = 1)


    # make a pivot table for visualization
    table = pd.pivot_table(res,values="Percent",index = ["Sample Type"],columns=["Visit Number"])
    
    # re-index the table based on the ordered list of visits
    if type(ct.cohort) == list:
        table = table.reindex(ct.visits[choose_cohort],axis = 1)
    else:
        table = table.reindex(ct.visits,axis = 1)


    # Put NaNs in cells we are not expecting samples

    if type(ct.cohort) == list:
        try:
            cohort_exceptions = ct.exceptions[choose_cohort]
            for except_visit in cohort_exceptions:
                table.loc[except_visit[0],except_visit[1]] = np.nan
        except:
            pass
    else:
        for except_visit in ct.exceptions:
            table.loc[except_visit[0],except_visit[1]] = np.nan
    
    return table,table_num_rec_vis

In [22]:

for trial in clinical_trials:
    if type(trial.cohort) == list:
        for chrt in trial.cohort:
            choose_cohort = chrt
            lv_data = get_lv_data(trial.studynum,chrt)
            rho_data = get_rho_data(trial.studynum,chrt)
            visit_info = get_visit_info()
            if trial.cohort[0] == chrt:
                result = create_result_df(rho_data,lv_data,visit_info)
            else:
                result = pd.concat([result,create_result_df(rho_data,lv_data,visit_info)])
            #table,table_num_rec_vis = pivot_table(rho_data,result,lv_data)
            #heatmap(table,table_num_rec_vis)
    else:
        lv_data = get_lv_data(trial.studynum,None)
        rho_data = get_rho_data(trial.studynum,None)
        visit_info = get_visit_info()
        result = create_result_df(rho_data,lv_data,visit_info)

        # rho_records,lv_data,visit_info,res
        # print(lv_data)
        # table,table_num_rec_vis = pivot_table(rho_data,result,lv_data)
        
        # heatmap(table,table_num_rec_vis)
    result

C:\Users\lserizawa\AppData\Local\Temp\ipykernel_30416\3730478193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(lv_query,cnxn)
C:\Users\lserizawa\AppData\Local\Temp\ipykernel_30416\3730478193.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(rho_query,cnxn)
C:\Users\lserizawa\AppData\Local\Temp\ipykernel_30416\3730478193.py:124: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  output_df = pd.read_sql(visit_query,cnxn)
C:\Users\lserizawa\AppData\Local\Temp\ipykernel_3041

,Study,Cohort,Visit Number,Visit Ordinal,DaysPostScreening,Sample Type,Number at least 1 collected,Number of recorded visits,Percent
0,ITN080AI,A,0A,15,0.0,Serum Clot,17,17,100
1,ITN080AI,A,0A,15,0.0,PBMC,17,17,100
2,ITN080AI,A,0A,15,0.0,Whole Blood\nTranscriptomics,16,17,94
3,ITN080AI,A,0A,15,0.0,Whole Blood\nEpigenetic,17,17,100
4,ITN080AI,A,0A,15,0.0,Urine Supernatant,17,17,100
...,...,...,...,...,...,...,...,...,...
85,ITN080AI,B,DVB,300,0.0,PBMC,3,3,100
86,ITN080AI,B,DVB,300,0.0,Whole Blood\nTranscriptomics,2,3,67
87,ITN080AI,B,DVB,300,0.0,Whole Blood\nEpigenetic,3,3,100
88,ITN080AI,B,DVB,300,0.0,Urine Supernatant,3,3,100
